In [1]:
! git clone https://gitlab.com/bigirqu/quran-qa-2023.git

Cloning into 'quran-qa-2023'...
remote: Enumerating objects: 922, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 922 (delta 169), reused 0 (delta 0), pack-reused 602
Receiving objects: 100% (922/922), 1.34 MiB | 6.92 MiB/s, done.
Resolving deltas: 100% (440/440), done.


In [2]:
! pip install farasa
! pip install transformers
! pip install farasapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.8 MB/s eta 0:00:00


#Import Libraries

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/')
import quranqa22_eval as eval

In [8]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import json
import sys
from tqdm import tqdm

#Load Data

In [9]:
train = pd.read_csv("/content/drive/MyDrive/QA_Quran/DataCsv/train.csv")
validation = pd.read_csv("/content/drive/MyDrive/QA_Quran/DataCsv/validation.csv")

In [10]:
train_dataset_jsonl = []
with open('/content/quran-qa-2023/Task-B/data/QQA23_TaskB_qrcd_v1.2_train_preprocessed.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

with open('/content/quran-qa-2023/Task-B/data/QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

In [11]:
len(train_dataset_jsonl)

1155

#Prepare Dataset For Prediction

In [12]:
data = pd.concat([train, validation])

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337 entries, 0 to 209
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pq_id       1337 non-null   object
 1   passage     1337 non-null   object
 2   question    1337 non-null   object
 3   answer      1337 non-null   object
 4   start_char  1337 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 62.7+ KB


In [14]:
data = data.filter(items=["pq_id","passage","question"])
data_unique_QA = data.drop_duplicates()

In [15]:
len(data_unique_QA)

1093

# Try Different Transformers Models

In [16]:
from transformers.utils.dummy_pt_objects import RemBertForSequenceClassification
def answer_question(row):
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result_row = pipe(question=row["question"], context=row["passage"], handle_impossible_answer=True, topk=5)
    return result_row

In [17]:
def Try_Transformer(tokenizer_name,model_name ):
  result = {}
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  # Apply the function to the "question" and "passage" columns of the DataFrame
  for i in tqdm(range(len(data_unique_QA))):
    result_part = answer_question(data_unique_QA.iloc[i])
    result.update({data_unique_QA.iloc[i]["pq_id"]:result_part})
  return result

In [18]:
all_scores = {}

# AraElectra-Arabic-SQuADv2-QA

In [19]:
tokenizer = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
model = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

  0%|          | 0/1093 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
  0%|          | 1/1093 [00:03<1:09:39,  3.83s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
100%|██████████| 1093/1093 [50:41<00:00,  2.78s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered ques

{'pRR': 0.3675792954920669, 'exact_match': 0.13852813852813853, 'f1': 0.33168953994722544}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


# camelbert-mix

In [20]:
tokenizer = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
model = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1093 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1093 [00:03<1:07:30,  3.71s/it]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pred

{'pRR': 0.1312436447678722, 'exact_match': 0.0, 'f1': 0.10510054656721556}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


# araelectra-base-generator

In [21]:
tokenizer = "aubmindlab/araelectra-base-generator"
model = "aubmindlab/araelectra-base-generator"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1093 [00:00<?, ?it/s]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1093 [00:01<27:38,  1.52s/it]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

{'pRR': 0.14096513250428488, 'exact_match': 0.0025974025974025974, 'f1': 0.11102393309775446}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#mdeberta-v3

In [22]:
tokenizer = "timpal0l/mdeberta-v3-base-squad2"
model = "timpal0l/mdeberta-v3-base-squad2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


100%|██████████| 1093/1093 [1:49:03<00:00,  5.99s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered ques

{'pRR': 0.2376515577304802, 'exact_match': 0.05281385281385281, 'f1': 0.1168438497608042}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#araelectra-base

In [23]:
tokenizer = "wissamantoun/araelectra-base-artydiqa"
model = "wissamantoun/araelectra-base-artydiqa"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1093/1093 [46:14<00:00,  2.54s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered questi

{'pRR': 0.35371607144635214, 'exact_match': 0.12380952380952381, 'f1': 0.3170142120161807}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#arap_qa_bert

In [24]:
tokenizer = "gfdgdfgdg/arap_qa_bert"
model = "gfdgdfgdg/arap_qa_bert"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1093/1093 [1:06:44<00:00,  3.66s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered ques

{'pRR': 0.09597852101286525, 'exact_match': 0.013852813852813853, 'f1': 0.03329277833737827}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#AraElectra-base-finetuned

In [25]:
tokenizer = "salti/AraElectra-base-finetuned-ARCD"
model = "salti/AraElectra-base-finetuned-ARCD"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1093/1093 [50:47<00:00,  2.79s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered questi

{'pRR': 0.3030058413540315, 'exact_match': 0.07965367965367966, 'f1': 0.27834489590910994}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#Bert_Arabic-SQuADv2-QA

In [26]:
tokenizer = "zohaib99k/Bert_Arabic-SQuADv2-QA"
model = "zohaib99k/Bert_Arabic-SQuADv2-QA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1093/1093 [48:37<00:00,  2.67s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered questi

{'pRR': 0.3675792954920669, 'exact_match': 0.13852813852813853, 'f1': 0.33168953994722544}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#AraELECTRA-discriminator

In [27]:
tokenizer = "Damith/AraELECTRA-discriminator-QuranQA"
model = "Damith/AraELECTRA-discriminator-QuranQA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1093/1093 [48:54<00:00,  2.69s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:14-15_251 will receive score 0.
Unanswered question 37:75-82_251 will receive score 0.
Unanswered question 11:45-48_251 will receive score 0.
Unanswered question 11:103-108_255 will receive score 0.
Unanswered question 38:41-44_255 will receive score 0.
Unanswered questi

{'pRR': 0.6650538712405222, 'exact_match': 0.5177489177489177, 'f1': 0.6506743087182008}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


#AraELECTRA-discriminator-SOQAL

In [29]:
tokenizer = "Damith/AraELECTRA-discriminator-SOQAL"
model = "Damith/AraELECTRA-discriminator-SOQAL"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

  0%|          | 0/1093 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
100%|██████████| 1093/1093 [50:44<00:00,  2.79s/it]
Unanswered question 28:85-88_323 will receive score 0.
Unanswered question 28:85-88_324 will receive score 0.
Unanswered question 65:1-3_338 will receive score 0.
Unanswered question 75:1-15_338 will receive score 0.
Unanswered question 24:1-3_358 will receive score 0.
Unanswered question 27:20-28_112 will receive score 0.
Unanswered question 2:247-248_112 will receive score 0.
Unanswered question 48:24-26_268 will receive score 0.
Unanswered question 6:71-73_110 will receive score 0.
Unanswered question 17:78-82_344 will receive score 0.
Unanswered question 97:1-5_344 will receive score 0.
Unanswered question 2:219-219_344 will receive score 0.
Unanswered question 29:

{'pRR': 0.2741659516678222, 'exact_match': 0.08398268398268398, 'f1': 0.20483090220400388}


Unanswered question 28:85-88_322 will receive score 0.
Unanswered question 24:1-3_322 will receive score 0.
Unanswered question 46:9-14_260 will receive score 0.
Unanswered question 51:7-14_260 will receive score 0.
Unanswered question 24:11-18_260 will receive score 0.
Unanswered question 86:11-17_384 will receive score 0.
Unanswered question 4:90-91_384 will receive score 0.
Unanswered question 39:27-31_384 will receive score 0.
Unanswered question 4:2-6_336 will receive score 0.
Unanswered question 33:59-59_336 will receive score 0.


In [30]:
filename = "scores.json"

with open(filename, "w") as json_file:
    json.dump(all_scores, json_file)

In [31]:
print("Number Of Transformers:",len(all_scores))

Number Of Transformers: 10
